In [ ]:
!wget https://course.ccs.neu.edu/cs6220/homework-1/data/combined_data_1.txt

--2023-01-26 02:56:11--  https://course.ccs.neu.edu/cs6220/homework-1/data/combined_data_1.txt
Resolving course.ccs.neu.edu (course.ccs.neu.edu)... 129.10.117.35
Connecting to course.ccs.neu.edu (course.ccs.neu.edu)|129.10.117.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 495032008 (472M) [text/plain]
Saving to: ‘combined_data_1.txt’

combined_data_1.txt 100%[===================>] 472.10M  35.4MB/s    in 14s     

2023-01-26 02:56:25 (33.8 MB/s) - ‘combined_data_1.txt’ saved [495032008/495032008]



# The Average Star Rating for All Users




In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 7.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=cbcb5ee8b2f972e56a11fd1aa25c71334ae0a4c369ab5f6703f358172003e642
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 23 not

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

# Question

Write a function that finds the average star rating over users. 

* Output: `(<USER>,<AVERAGE RATINGS>)`

(<USER>, Float(Star Rating), 1.0)

REDUCE_BY_KEY (Key USER, SUM(Star Ratings), SUM(1.0's))

USER 1, (Star Rating 1, 1)
USER 1, (Star Rating 2, 1)

(<ABCDE>, 25.0, 5.0)

MAP --> (USER, SUM_STAR_RATING / SUM_1.0s)

(<ABCDE>, 5.0)

In [ ]:
!head -n 10000 combined_data_1.txt > short_data.txt

In [ ]:
lines = sc.textFile("short_data.txt", 1).filter(lambda x: ":" not in x).map(lambda x: x.split(","))
user_star_average = lines.map(lambda x: (x[0], (float(x[1]), 1.0))) .reduceByKey(lambda x, y: (x[0] + y[0], x[1]+y[1])).map(lambda x: (x[0], x[1][0] / x[1][1]))
user_star_average.collect()


[('1488844', 3.5),
 ('822109', 5.0),
 ('885013', 4.5),
 ('30878', 2.5),
 ('823519', 2.0),
 ('893988', 3.0),
 ('124105', 4.0),
 ('1248029', 3.0),
 ('1842128', 4.0),
 ('2238063', 3.0),
 ('1503895', 4.0),
 ('2207774', 5.0),
 ('2590061', 3.0),
 ('2442', 3.0),
 ('543865', 4.0),
 ('1209119', 4.0),
 ('804919', 4.0),
 ('1086807', 3.0),
 ('1711859', 4.0),
 ('372233', 5.0),
 ('1080361', 3.0),
 ('1245640', 3.0),
 ('558634', 4.0),
 ('2165002', 4.0),
 ('1181550', 3.0),
 ('1227322', 4.5),
 ('427928', 4.0),
 ('814701', 5.0),
 ('808731', 4.0),
 ('662870', 5.0),
 ('337541', 5.0),
 ('786312', 2.6666666666666665),
 ('1133214', 4.0),
 ('1537427', 4.0),
 ('1209954', 5.0),
 ('2381599', 3.0),
 ('525356', 2.3333333333333335),
 ('1910569', 4.0),
 ('2263586', 4.0),
 ('2421815', 2.0),
 ('1009622', 1.0),
 ('1481961', 2.0),
 ('401047', 2.3333333333333335),
 ('2179073', 3.0),
 ('1434636', 3.0),
 ('93986', 4.0),
 ('1308744', 5.0),
 ('2647871', 4.5),
 ('1905581', 5.0),
 ('2508819', 3.0),
 ('1578279', 1.0),
 ('1159695

# Distribution of Star Ratings

Find the distribution of number of users over star ratings.

* Output: `(<Number-Stars>, <TOTAL-NUMBER-USERS>)`

( lines[1], 1 )


In [ ]:
lines.map( lambda x: ( x[1], 1 )).reduceByKey(lambda x, y: x + y).collect()

[('3', 2450), ('5', 2190), ('4', 2926), ('2', 1260), ('1', 1166)]